In [1]:
import os
import sys
# Add the src directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)
from src.data_handling.historical_data_handler import SingleSymbolDataHandler
from src.feature_engineering.feature_extractor import SingleSymbolFeatureExtractor
%cd $project_root

/Users/zhaoyub/Documents/Tradings/tradebotv3


/Users/zhaoyub/Library/Python/3.12/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%load_ext autoreload

In [7]:
import pandas as pd
import numpy as np

In [31]:
datahandler = SingleSymbolDataHandler('BTCUSDT')
inter, begin, end = '15m', '2023-01-01', '2024-09-24'
datahandler.load_data(inter, begin, end)
BTC_indicator = SingleSymbolFeatureExtractor('BTCUSDT', datahandler)
BTC_indicator.load_full_range()
indicators = BTC_indicator.indicators[['rsi', 'macd', 'stoch_k', 'bollinger_upper', 'atr', 'obv', 'vwap', 'adx']]
indicators = pd.concat([indicators,datahandler.cleaned_data[['close', 'volume']]],axis= 1)

In [32]:
indicators['rsi_variance'] = indicators['rsi'].rolling(20).std().bfill().ffill()

In [33]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
indicators_scaled = pd.DataFrame(scaler.fit_transform(indicators), columns=indicators.columns)

In [34]:
correlation_matrix = indicators_scaled.corr()
correlation_matrix['rsi_variance'].sort_values(ascending=False)

rsi_variance       1.000000
adx                0.290782
atr                0.194216
volume             0.186044
obv                0.011247
bollinger_upper    0.001096
vwap              -0.008054
close             -0.008255
rsi               -0.008459
stoch_k           -0.010975
macd              -0.031835
Name: rsi_variance, dtype: float64